# cross session之訓練與預測(串聯2個相鄰session, source&target)

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import r2_score
from dataPreprocess import read_dataset

In [ ]:
sourceSessionIndex = 15
targetSessionIndex = 16

In [ ]:
folderPath = './data'
fileList = sorted([i for i in os.listdir(folderPath) if i.endswith('.pkl.zip')])

In [ ]:
sourceSession = fileList[sourceSessionIndex]
targetSession = fileList[targetSessionIndex]

In [ ]:
sourceDataset = read_dataset(os.path.join(folderPath, sourceSession), ['velocity_x', 'velocity_y'])
targetDataset = read_dataset(os.path.join(folderPath, targetSession), ['velocity_x', 'velocity_y'])

In [ ]:
from model_tf import simpleDecodeModel

model = simpleDecodeModel()
model.compile(optimizer='adam', loss='mse')

In [ ]:
TRAIN_COUNT = 5000

train_x = np.concatenate((targetDataset['m1'][:TRAIN_COUNT, -5:, :], sourceDataset['m1'][:TRAIN_COUNT, -5:, :]), axis=0)
train_y = np.concatenate((targetDataset['movement'][:TRAIN_COUNT], sourceDataset['movement'][:TRAIN_COUNT]), axis=0)
test_x = targetDataset['m1'][TRAIN_COUNT:, -5:, :]
test_y = targetDataset['movement'][TRAIN_COUNT:]

test_timestamp = targetDataset['timestamp'][TRAIN_COUNT:]

In [ ]:
# z-score
train_y_mean, train_y_std = np.mean(train_y, axis=0), np.std(train_y, axis=0)
train_y = (train_y - train_y_mean) / train_y_std
test_y = (test_y - train_y_mean) / train_y_std

In [ ]:
# train
model.fit(x=train_x, y=train_y, batch_size=128, epochs=100, shuffle=True, verbose=0)

In [ ]:
# pred
pred = model.predict(x=test_x)

In [ ]:
# 還原
pred = pred * train_y_std + train_y_mean
test_y = test_y * train_y_std + train_y_mean

In [ ]:
# 計算R2
r2 = [r2_score(y_pred=pred[:, i], y_true=test_y[:, i]) for i in range(pred.shape[-1])]
print('vx:', r2[0])
print('vy:', r2[1])

In [ ]:
# 輸出
result = pd.DataFrame(columns=[])
axis = ['x', 'y']
for n, a in enumerate(axis):
    result = result.append(pd.DataFrame({
        'timestamp': test_timestamp,
        'value': test_y[:, n],
        'type': 'groundtruth',   
        'movement': 'velocity',
        'axis': a,          
        'test_on': targetSessionIndex,
        'r2': r2[n]     
    }))
    result = result.append(pd.DataFrame({
        'timestamp': test_timestamp,
        'value': pred[:, n],
        'type': 'prediction',   
        'movement': 'velocity',
        'axis': a, 
        'test_on': targetSessionIndex,  
        'r2': r2[n]           
    }))
result.reset_index(inplace=True, drop=True)

result.to_csv('./results/trainOnMix.csv', index=False, header=True)